In [ ]:
%pip install pandas scikit-learn numpy Streamlit llama-index langchain

Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following versions that require a different python version: 0.55.2 Requires-Python <3.5; 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement LlamaIndex (from versions: none)
ERROR: No matching distribution found for LlamaIndex


In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors


In [ ]:
# Загрузка и обработка датасета
data = pd.read_csv('data/music_dataset.csv')
data

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
0,0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,95,0.000598,0.063746,0.000598,...,0.380299,0.117175,0.357739,0.454119,0.997992,0.901822,0.339448,0.137110,sadness,1.000000
1,4,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,51,0.035537,0.096777,0.443435,...,0.001284,0.001284,0.331745,0.647540,0.954819,0.000002,0.325021,0.263240,world/life,1.000000
2,6,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,24,0.002770,0.002770,0.002770,...,0.002770,0.225422,0.456298,0.585288,0.840361,0.000000,0.351814,0.139112,music,1.000000
3,10,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54,0.048249,0.001548,0.001548,...,0.225889,0.001548,0.686992,0.744404,0.083935,0.199393,0.775350,0.743736,romantic,1.000000
4,12,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48,0.001350,0.001350,0.417772,...,0.068800,0.001350,0.291671,0.646489,0.975904,0.000246,0.597073,0.394375,romantic,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28367,82447,mack 10,10 million ways,2019,hip hop,cause fuck leave scar tick tock clock come kno...,78,0.001350,0.001350,0.001350,...,0.065664,0.001350,0.889527,0.759711,0.062549,0.000000,0.751649,0.695686,obscene,0.014286
28368,82448,m.o.p.,ante up (robbin hoodz theory),2019,hip hop,minks things chain ring braclets yap fame come...,67,0.001284,0.001284,0.035338,...,0.001284,0.001284,0.662082,0.789580,0.004607,0.000002,0.922712,0.797791,obscene,0.014286
28369,82449,nine,whutcha want?,2019,hip hop,get ban get ban stick crack relax plan attack ...,77,0.001504,0.154302,0.168988,...,0.001504,0.001504,0.663165,0.726970,0.104417,0.000001,0.838211,0.767761,obscene,0.014286
28370,82450,will smith,switch,2019,hip hop,check check yeah yeah hear thing call switch g...,67,0.001196,0.001196,0.001196,...,0.001196,0.001196,0.883028,0.786888,0.007027,0.000503,0.508450,0.885882,obscene,0.014286


In [6]:
# Выбираем признаки
audio_features = ['danceability', 'loudness', 'acousticness', 'instrumentalness', 'valence', 'energy']
lyric_features = ['dating', 'violence', 'world/life', 'night/time', 'shake the audience', 'family/gospel', 
                  'romantic', 'communication', 'obscene', 'music', 'movement/places', 'light/visual perceptions', 
                  'family/spiritual', 'like/girls', 'sadness', 'feelings']

In [7]:
# Масштабируем числовые признаки
scaler = StandardScaler()
scaled_audio_features = scaler.fit_transform(data[audio_features])
scaled_lyric_features = scaler.fit_transform(data[lyric_features])

In [8]:
# Комбинируем признаки
features = pd.DataFrame(scaled_audio_features, columns=audio_features).join(pd.DataFrame(scaled_lyric_features, columns=lyric_features))

In [10]:
# Снижение размерности для эффективности
pca = PCA(n_components=20)
reduced_features = pca.fit_transform(features)


# Создание модели для поиска похожих треков

In [11]:
model = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
model.fit(reduced_features)

NearestNeighbors(algorithm='kd_tree', n_neighbors=10)

In [12]:
def recommend_tracks(track_name):
    # Находим индекс трека
    track_id = data.index[data['track_name'] == track_name].tolist()[0]
    distances, indices = model.kneighbors([reduced_features[track_id]])
    recommendations = data.iloc[indices[0]]
    return recommendations[['artist_name', 'track_name', 'genre', 'release_date']].to_dict(orient='records')

# Рекомендации с Retrieval-Augmented Generation (RAG)

In [15]:
from langchain.prompts import PromptTemplate
from langchain.chat_models.gigachat import GigaChat

import os

In [17]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)

In [24]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [ ]:
from langchain.schema import HumanMessage, SystemMessage

messages = [
    SystemMessage(
        content="Ты эксперт по музыке, помогаешь подбирать музыку по контексту."
    )
]

In [27]:
def get_rag_recommendations(track_name):
    documents = SimpleDirectoryReader(
        "data"
    ).load_data()  
    index = VectorStoreIndex.from_documents(documents)
    retriever = index.as_retriever(verbose=True)
    context = retriever.retrieve(track_name)
    prompt = f"Рекомендовать похожие треки для {track_name} на основе данных: {context}"
    
    messages.append(HumanMessage(content=prompt))
    
    res = giga(messages)
    messages.append(res)
    print("Bot: ", res.content)
    

In [29]:
get_rag_recommendations("Du hast — Rammstein")

RuntimeError: no validator found for <class 'langchain.chains.llm.LLMChain'>, see `arbitrary_types_allowed` in Config